<a href="https://colab.research.google.com/github/priyansuapk/ml-lab-exam/blob/main/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

class SVM_Dual_Kernel:
    def __init__(self, C=1.0, learning_rate=0.001, n_iters=1000, kernel='linear', degree=3, gamma=0.5):
        self.C = C
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.kernel = kernel       # Specifies which kernel to use: 'linear', 'rbf', or 'polynomial'
        self.degree = degree       # Degree for polynomial kernel
        self.gamma = gamma         # Gamma for RBF kernel
        self.alpha = None          # Lagrange multipliers
        self.w = None              # Weight vector for linear kernel
        self.b = None              # Bias term
        self.X_train = None        # Training data
        self.y_train = None        # Training labels

    # Method to compute the kernel matrix based on the selected kernel
    def compute_kernel_matrix(self, X, Y=None):
        if Y is None:
            Y = X  # Use X for both if Y is not provided (e.g., for training)
        if self.kernel == 'linear':
            return np.dot(X, Y.T)  # Linear kernel: simple dot product
        elif self.kernel == 'rbf':
            # RBF kernel: exp(-gamma * ||x - y||^2)
            sq_dists = np.sum(X**2, axis=1).reshape(-1, 1) + np.sum(Y**2, axis=1) - 2 * np.dot(X, Y.T)
            return np.exp(-self.gamma * sq_dists)
        elif self.kernel == 'polynomial':
            # Polynomial kernel: (x · y + 1) ^ degree
            return (np.dot(X, Y.T) + 1) ** self.degree
        else:
            raise ValueError("Unknown kernel type")

    # Fit method to train the SVM model
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y.astype(float)
        n_samples, n_features = X.shape
        K = self.compute_kernel_matrix(X)  # Compute the kernel matrix for training

        # Initialize Lagrange multipliers (alpha) to zero
        self.alpha = np.zeros(n_samples)

        # Perform gradient ascent on the dual objective function
        for _ in range(self.n_iters):
            for i in range(n_samples):
                gradient = 1 - np.sum(self.alpha * self.y_train * K[:, i])
                self.alpha[i] += self.learning_rate * gradient * self.y_train[i]

                # Clip alpha values to satisfy constraints: 0 <= alpha_i <= C
                self.alpha[i] = min(max(self.alpha[i], 0), self.C)

            # Ensure sum(alpha_i * y_i) = 0 by adjusting alpha values if needed
            sum_alpha_y = np.sum(self.alpha * self.y_train)
            if sum_alpha_y != 0:
                adjustment = sum_alpha_y / n_samples
                self.alpha -= adjustment * self.y_train

        # Compute the bias term (b) using support vectors
        support_vectors = self.alpha > 1e-5
        self.b = np.mean(self.y_train[support_vectors] - np.dot(K[support_vectors], self.alpha * self.y_train))

    # Predict method to classify new samples based on the learned model
    def predict(self, X):
      K_test = self.compute_kernel_matrix(self.X_train, X)
      # Reshape (self.alpha * self.y_train) to (1, -1) for correct matrix multiplication
      decision = np.dot(self.alpha * self.y_train, K_test) + self.b
      return np.sign(decision).flatten()  # Flatten to return a 1D array of predictions








In [ ]:
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split

# Load Iris dataset and filter for two classes only (binary classification)
iris = load_iris()
X_iris, y_iris = iris.data, iris.target
# Filter to only include classes 0 and 1
iris_filter = y_iris < 2
X_iris, y_iris = X_iris[iris_filter], y_iris[iris_filter]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_iris, y_iris, test_size=0.3, random_state=42)

# Load Breast Cancer dataset
cancer = load_breast_cancer()
X_cancer, y_cancer = cancer.data, cancer.target
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_cancer, y_cancer, test_size=0.3, random_state=42)

# Define datasets dictionary for use in SVM training
datasets = {
    "Dataset 1": (X_train1, X_test1, y_train1, y_test1),
    "Dataset 2": (X_train2, X_test2, y_train2, y_test2),
}


In [ ]:
# Define the datasets
datasets = {
    "Dataset 1": (X_train1, X_test1, y_train1, y_test1),
    "Dataset 2": (X_train2, X_test2, y_train2, y_test2),
}

# Define kernels to use
kernels = ['linear', 'rbf', 'polynomial']

# Initialize an empty dictionary to store results by dataset
results = {dataset_name: {} for dataset_name in datasets.keys()}

# Function to calculate accuracy
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Training and evaluation loop
for dataset_name, (X_train, X_test, y_train, y_test) in datasets.items():
    for kernel in kernels:
        # Initialize the SVM with the current kernel
        clf = SVM_Dual_Kernel(kernel=kernel, n_iters=1000, gamma=0.5, degree=3)
        clf.fit(X_train, y_train)  # Train the model

        # Make predictions and calculate accuracy
        predictions = clf.predict(X_test)
        acc = accuracy(y_test, predictions)

        # Store result in a structured format
        results[dataset_name][kernel] = acc

# Print the results in the requested format
print("Results Summary:")
for dataset_name, kernel_accuracies in results.items():
    for kernel, acc in kernel_accuracies.items():
        print(f"{dataset_name} with {kernel.capitalize()} Kernel Accuracy: {acc:.2f}")


Results Summary:
Dataset 1 with Linear Kernel Accuracy: 0.43
Dataset 1 with Rbf Kernel Accuracy: 0.43
Dataset 1 with Polynomial Kernel Accuracy: 0.30
Dataset 2 with Linear Kernel Accuracy: 0.11
Dataset 2 with Rbf Kernel Accuracy: 0.63
Dataset 2 with Polynomial Kernel Accuracy: 0.15
